In [1]:
import pandas as pd
import py2neo
import urllib.request as urllib
from py2neo import Graph,Node,Relationship
import xlrd

In [13]:
esc_folder = "eurovision_esc_2014_2015_2016_2017/"

#load all excel sheets in an "efficient" way
ESC2014_2015_2016 = ["ESC2014_SF1","ESC2014_SF2","ESC2014_GF","ESC2015_SF1","ESC2015_SF2","ESC2015_GF","ESC2016_SF1","ESC2016_SF2","ESC2016_GF"]
result_types = ["Voting", "Jury members"]
for file in ESC2014_2015_2016:
    for rt in result_types:
        globals()[(file + '_' + rt).replace(" ", "")] = pd.read_excel("data/" + esc_folder + file + ".xls",sheet_name = rt,header=1)


ESC2017 = ["ESC2017_SF1", "ESC2017_SF2", "ESC2017_GF"]
result_types = ["Televoting" , "Jury voting", "Combined"]
for file in ESC2017:
    for rt in result_types:
        globals()[(file + "_" + rt).replace(" ", "")] = pd.read_excel("data/" + esc_folder + file + ".xlsx", sheet_name = rt, header =1)


ESC2017_juro = ["ESC2017_jurorresults_SF1","ESC2017_jurorresults_SF2","ESC2017_jurorresults_GF"]
for file in ESC2017_juro:
    globals()[(file).replace(" ", "")] = pd.read_excel("data/" + esc_folder + file + ".xlsx", header=1)

all_variables = dir()
print("Dataframes: ")
all_variables[0:33]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

['ESC2014_2015_2016',
 'ESC2014_GF_Jurymembers',
 'ESC2014_GF_Voting',
 'ESC2014_SF1_Jurymembers',
 'ESC2014_SF1_Voting',
 'ESC2014_SF2_Jurymembers',
 'ESC2014_SF2_Voting',
 'ESC2015_GF_Jurymembers',
 'ESC2015_GF_Voting',
 'ESC2015_SF1_Jurymembers',
 'ESC2015_SF1_Voting',
 'ESC2015_SF2_Jurymembers',
 'ESC2015_SF2_Voting',
 'ESC2016_GF_Jurymembers',
 'ESC2016_GF_Voting',
 'ESC2016_SF1_Jurymembers',
 'ESC2016_SF1_Voting',
 'ESC2016_SF2_Jurymembers',
 'ESC2016_SF2_Voting',
 'ESC2017',
 'ESC2017_GF_Combined',
 'ESC2017_GF_Juryvoting',
 'ESC2017_GF_Televoting',
 'ESC2017_SF1_Combined',
 'ESC2017_SF1_Juryvoting',
 'ESC2017_SF1_Televoting',
 'ESC2017_SF2_Combined',
 'ESC2017_SF2_Juryvoting',
 'ESC2017_SF2_Televoting',
 'ESC2017_juro',
 'ESC2017_jurorresults_GF',
 'ESC2017_jurorresults_SF1',
 'ESC2017_jurorresults_SF2']

In [2]:
#connect to database
f=open("authentication/neo4j.txt","r")
lines=f.readlines()
user=lines[0].replace('\n','')
pwd=lines[1].replace('\n','')
http_port = lines[2]
f.close()

graph = Graph(http_port, auth=(user, pwd))

In [7]:
# laod data in neo4j database
fp = urllib.urlopen("https://goz39a.s3.eu-central-1.amazonaws.com/eurovision/eurovision.txt")
mybytes = fp.read()
data = mybytes.decode("utf8")
fp.close()
graph.run(data)
graph.run("MATCH (n) RETURN count(*)")

count(*)
46


In [6]:
def clear_graph(): 
    return (graph.run("MATCH (n)-[r]->() DELETE r"), graph.run("MATCH (n) DELETE n"))
    
clear_graph()

((No data), (No data))